In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

In [2]:
## get the variables

PATH_1=os.path.join('..','..','..','Weather_Data','ERA5','2019','era_2019.nc')
humidity_data = xr.open_dataset(PATH_1) ## variables
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2019-01-01 ... 2019-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
    expver          (valid_time) <U4 47kB ...
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T19:53 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
humidity_data= humidity_data.drop_vars('number') ## not sure what these two are
humidity_data= humidity_data.drop_vars('expver')
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 23kB 2019-01-01 ... 2019-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T19:53 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
humidity_data.to_dataframe()

r         q
valid_time          pressure_level latitude longitude                     
2019-01-01 00:00:00 1000.0         49.25    -124.85    75.108887  0.003509
                                            -124.60    71.665527  0.003316
                                            -124.35    69.118652  0.003211
                                            -124.10    67.479980  0.003220
                                            -123.85    68.188965  0.003246
...                                                          ...       ...
2019-12-31 21:00:00 1000.0         24.50    -68.10     75.695267  0.013292
                                            -67.85     72.455032  0.012975
                                            -67.60     69.144485  0.012526
                                            -67.35     66.603470  0.012066
                                            -67.10     64.253860  0.011624

[67744000 rows x 2 columns]

In [5]:
## resample the data to 1 year average

year_avg = humidity_data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2019-12-31
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T19:53 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
year_avg.to_dataframe()

r         q
valid_time pressure_level latitude longitude                     
2019-12-31 1000.0         49.25    -124.85    81.350853  0.006222
                                   -124.60    79.078247  0.006218
                                   -124.35    77.065590  0.006150
                                   -124.10    74.702286  0.006082
                                   -123.85    71.468338  0.006067
...                                                 ...       ...
                          24.50    -68.10     78.145218  0.014875
                                   -67.85     78.071777  0.014851
                                   -67.60     78.037300  0.014831
                                   -67.35     78.009789  0.014810
                                   -67.10     78.003654  0.014792

[23200 rows x 2 columns]

In [7]:
year_avg = year_avg.squeeze(dim="pressure_level") ## drop the pressure level dimension
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, latitude: 100, longitude: 232)
Coordinates:
    pressure_level  float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2019-12-31
Data variables:
    r               (valid_time, latitude, longitude) float32 93kB 81.35 ... ...
    q               (valid_time, latitude, longitude) float32 93kB 0.006222 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T19:53 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
SHAPE_PATH=os.path.join('..','..','..','Shapefiles','county_shapefiles','2019_county_shapefile','cb_2019_us_county_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,48,081,01383826,0500000US48081,48081,Coke,06,2361153195,42331832,"POLYGON ((-100.82497 31.74941, -100.82415 31.8..."
1,48,273,01383922,0500000US48273,48273,Kleberg,06,2282572445,541041659,"MULTIPOLYGON (((-97.31780 27.49456, -97.31590 ..."
2,48,203,01383887,0500000US48203,48203,Harrison,06,2331138836,40651525,"POLYGON ((-94.70215 32.45618, -94.70197 32.467..."
3,48,223,01383897,0500000US48223,48223,Hopkins,06,1987629163,65639829,"POLYGON ((-95.86333 33.04989, -95.86302 33.065..."
4,48,033,01383802,0500000US48033,48033,Borden,06,2324366073,22297606,"POLYGON ((-101.69128 32.96184, -101.55743 32.9..."
...,...,...,...,...,...,...,...,...,...,...
3228,20,119,00485024,0500000US20119,20119,Meade,06,2533222397,3258258,"POLYGON ((-100.65264 37.41669, -100.65264 37.4..."
3229,38,049,01035305,0500000US38049,38049,McHenry,06,4853487956,97323497,"POLYGON ((-101.05991 48.38598, -101.05988 48.4..."
3230,72,119,01804540,0500000US72119,72119,Río Grande,13,157008568,74876349,"POLYGON ((-65.87666 18.36314, -65.87598 18.364..."
3231,46,021,01266974,0500000US46021,46021,Campbell,06,1900214169,96243697,"POLYGON ((-100.49935 45.94363, -100.46284 45.9..."


In [9]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-100.82497 31.74941, -100.82415 31.8...
1       MULTIPOLYGON (((-97.31780 27.49456, -97.31590 ...
2       POLYGON ((-94.70215 32.45618, -94.70197 32.467...
3       POLYGON ((-95.86333 33.04989, -95.86302 33.065...
4       POLYGON ((-101.69128 32.96184, -101.55743 32.9...
                              ...                        
3228    POLYGON ((-100.65264 37.41669, -100.65264 37.4...
3229    POLYGON ((-101.05991 48.38598, -101.05988 48.4...
3230    POLYGON ((-65.87666 18.36314, -65.87598 18.364...
3231    POLYGON ((-100.49935 45.94363, -100.46284 45.9...
3232    POLYGON ((-96.45277 44.19689, -96.45267 44.254...
Name: geometry, Length: 3233, dtype: geometry

In [10]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0  -100.824969  31.749414
     1  -100.824151  31.812061
     2  -100.823910  31.814215
     3  -100.824312  31.816345
     4  -100.823738  31.850077
...             ...        ...
3232 89  -96.322919  44.197382
     90  -96.331585  44.197349
     91  -96.450323  44.196978
     92  -96.452774  44.196780
     93  -96.452774  44.196895

[1040851 rows x 2 columns]

In [11]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.  
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            county_df: Dataframe consisting the values of the variables interpolated in the county.
    ''' 
    longitude= lat_lon.loc[(var), 'x']  # get the longitude 
    extract_val=len(longitude)        ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    
    latitude= lat_lon.loc[(var), 'y']   # get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
    year_avg_finer
    
    df = year_avg_finer.squeeze(dim="valid_time") ## remove the valid_time dimension, note that this will appear as a column
                                                  ## after removed

    ## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=df.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)

    initial_df=pd.concat(get_vals)
    initial_df=initial_df.drop('valid_time', axis=1) ## remove the valid_time column 
    
    mean=initial_df.mean() ## find the average of the 100 (or less) values
    average_df = pd.DataFrame(mean).T 

    county_df= average_df.convert_dtypes() ## the dtype of the mean values will be object, so convert that to float
    county_df
    
    return county_df

In [12]:
%%time

## include all counties

df_list=[]

for i in range(0,3233): ## A total of 3232 counties. index in shape file starts from 0 and ends in 3232
    try:
        df_list.append(single_county(i))
    except:
        pass

CPU times: total: 6min 8s
Wall time: 6min 17s


In [13]:
concatenate_df=pd.concat(df_list)
final_df=concatenate_df.reset_index(drop=True)
final_df = final_df.drop('pressure_level', axis=1)
final_df

,r,q
0,54.485853,0.008437
1,79.564177,0.013977
2,67.432572,0.010322
3,69.713038,0.010414
4,51.860676,0.007779
...,...,...
3228,56.911892,0.007022
3229,73.71036,0.004817
3230,<NA>,<NA>
3231,74.437042,0.005462


In [14]:
concatenate_df=pd.concat(df_list)
final_df=concatenate_df.reset_index(drop=True)
final_df

,r,q,pressure_level
0,54.485853,0.008437,1000
1,79.564177,0.013977,1000
2,67.432572,0.010322,1000
3,69.713038,0.010414,1000
4,51.860676,0.007779,1000
...,...,...,...
3228,56.911892,0.007022,1000
3229,73.71036,0.004817,1000
3230,<NA>,<NA>,1000
3231,74.437042,0.005462,1000


In [15]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

STATEFP       object
COUNTYFP      object
COUNTYNS      object
AFFGEOID      object
GEOID         object
NAME          object
LSAD          object
ALAND          int64
AWATER         int64
geometry    geometry
fips           int32
dtype: object

In [16]:
county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','AFFGEOID','GEOID','LSAD','ALAND','AWATER'], axis=1)
county_gdf

,NAME,geometry,fips
0,Coke,"POLYGON ((-100.82497 31.74941, -100.82415 31.8...",48081
1,Kleberg,"MULTIPOLYGON (((-97.31780 27.49456, -97.31590 ...",48273
2,Harrison,"POLYGON ((-94.70215 32.45618, -94.70197 32.467...",48203
3,Hopkins,"POLYGON ((-95.86333 33.04989, -95.86302 33.065...",48223
4,Borden,"POLYGON ((-101.69128 32.96184, -101.55743 32.9...",48033
...,...,...,...
3228,Meade,"POLYGON ((-100.65264 37.41669, -100.65264 37.4...",20119
3229,McHenry,"POLYGON ((-101.05991 48.38598, -101.05988 48.4...",38049
3230,Río Grande,"POLYGON ((-65.87666 18.36314, -65.87598 18.364...",72119
3231,Campbell,"POLYGON ((-100.49935 45.94363, -100.46284 45.9...",46021


In [17]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,r,q,pressure_level
0,Coke,"POLYGON ((-100.82497 31.74941, -100.82415 31.8...",48081,54.485853,0.008437,1000
1,Kleberg,"MULTIPOLYGON (((-97.31780 27.49456, -97.31590 ...",48273,79.564177,0.013977,1000
2,Harrison,"POLYGON ((-94.70215 32.45618, -94.70197 32.467...",48203,67.432572,0.010322,1000
3,Hopkins,"POLYGON ((-95.86333 33.04989, -95.86302 33.065...",48223,69.713038,0.010414,1000
4,Borden,"POLYGON ((-101.69128 32.96184, -101.55743 32.9...",48033,51.860676,0.007779,1000
...,...,...,...,...,...,...
3228,Meade,"POLYGON ((-100.65264 37.41669, -100.65264 37.4...",20119,56.911892,0.007022,1000
3229,McHenry,"POLYGON ((-101.05991 48.38598, -101.05988 48.4...",38049,73.71036,0.004817,1000
3230,Río Grande,"POLYGON ((-65.87666 18.36314, -65.87598 18.364...",72119,<NA>,<NA>,1000
3231,Campbell,"POLYGON ((-100.49935 45.94363, -100.46284 45.9...",46021,74.437042,0.005462,1000


In [18]:
county_df=merged_df.dropna()
county_df

,NAME,geometry,fips,r,q,pressure_level
0,Coke,"POLYGON ((-100.82497 31.74941, -100.82415 31.8...",48081,54.485853,0.008437,1000
1,Kleberg,"MULTIPOLYGON (((-97.31780 27.49456, -97.31590 ...",48273,79.564177,0.013977,1000
2,Harrison,"POLYGON ((-94.70215 32.45618, -94.70197 32.467...",48203,67.432572,0.010322,1000
3,Hopkins,"POLYGON ((-95.86333 33.04989, -95.86302 33.065...",48223,69.713038,0.010414,1000
4,Borden,"POLYGON ((-101.69128 32.96184, -101.55743 32.9...",48033,51.860676,0.007779,1000
...,...,...,...,...,...,...
3227,Dodge,"POLYGON ((-89.00953 43.32347, -89.00935 43.327...",55027,72.376481,0.006111,1000
3228,Meade,"POLYGON ((-100.65264 37.41669, -100.65264 37.4...",20119,56.911892,0.007022,1000
3229,McHenry,"POLYGON ((-101.05991 48.38598, -101.05988 48.4...",38049,73.71036,0.004817,1000
3231,Campbell,"POLYGON ((-100.49935 45.94363, -100.46284 45.9...",46021,74.437042,0.005462,1000


In [19]:
type(county_df)

geopandas.geodataframe.GeoDataFrame

In [20]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,r,q,pressure_level
0,Coke,48081,54.485852749952187,0.008436916165741,1000
1,Kleberg,48273,79.564176758154431,0.013977100669835,1000
2,Harrison,48203,67.432572308585065,0.010321682731408,1000
3,Hopkins,48223,69.713038326993370,0.010414466388858,1000
4,Borden,48033,51.860676142439551,0.007779232032411,1000
...,...,...,...,...,...
3227,Dodge,55027,72.376480563887512,0.006110887899761,1000
3228,Meade,20119,56.911892456876430,0.007022352507299,1000
3229,McHenry,38049,73.710359531707212,0.004816998782510,1000
3231,Campbell,46021,74.437041917166368,0.005461858279119,1000


In [21]:
county_var = county_var.sort_values(by='fips')
county_var

,NAME,fips,r,q,pressure_level
263,Autauga,1001,66.401566749177221,0.010217151184276,1000
222,Baldwin,1003,73.412622929325138,0.012133176279313,1000
166,Barbour,1005,68.309389285562986,0.010652697585906,1000
2526,Bibb,1007,67.297093059624686,0.010027599597121,1000
894,Blount,1009,68.275826389159405,0.009733986103932,1000
...,...,...,...,...,...
1826,Sweetwater,56037,59.683907659937070,0.003651965784675,1000
1163,Teton,56039,69.544237907324401,0.003680952572612,1000
1011,Uinta,56041,59.699260933286254,0.003632564658297,1000
2158,Washakie,56043,58.175270067637896,0.004117935850445,1000


In [22]:
county_var.to_pickle('era_2019.pkl')